In [ ]:
import librosa
import numpy as np
import glob
import os
import cv2
import pandas as pd
import keras
from keras import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [ ]:
# Membuat daftar folder yang akan digunakan
sample = ['OAF_sad', 'OAF_happy', 'OAF_angry']
# Membuat list untuk menampung spektogram dan label
spektogram, labels = [], []

In [ ]:
# Looping untuk melakukan ekstraksi fitur pada setiap file suara
for folder in sample:
    for file in glob.glob(folder + '/*.wav'):
# Membaca file suara menggunakan library librosa
        y, sr = librosa.load(file)

In [ ]:
# Mengambil nilai spektrogram dari file suara
spec = np.abs(librosa.stft(y))
spec = librosa.amplitude_to_db(spec, ref=np.max)

# Mengubah ukuran spektrogram menjadi 64x64 piksel dan melakukan flattening
spec = cv2.resize(spec, (64, 64))
spec = spec.flatten()

# Menambahkan spektrogram dan label ke dalam list
spektogram.append(spec)
labels.append(folder)

# Membuat dataframe dari spektrogram dan label
data = pd.DataFrame(spektogram)
data['Label'] = labels

# Melakukan label encoding pada label
y = data['Label'].values
le = LabelEncoder()
Y = le.fit_transform(y)
Y = to_categorical(Y, 3)

# Memisahkan data menjadi data train dan data test
X = data.drop(columns=['Label'])
X = X.values
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,
random_state=42)



In [ ]:
# Membuat model neural network dengan keras
model = Sequential()
model.add(Dense(2048, input_dim=4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001),
metrics=['accuracy'])

# Melatih model dengan data train
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50,
batch_size=32)


In [ ]:
# Memprediksi label dari data test dan menghitung akurasi
y_pred = model.predict_classes(X_test)
y_true = [np.argmax(x) for x in y_test]
accuracy = np.mean(y_pred == y_true)
print('Accuracy:', accuracy)
